For my solo exploration, I want to look at different countries over time and compare their totla consumpiton of electric power in thousands of kW per year. I also want to show how much of that total consumption is produceed by means of nuclear power.   

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
total_electricity = pd.read_csv('../data/total_electricity.csv')
nuclear_electricity = pd.read_csv('../data/nuclear_electricity.csv')

In [3]:
total_electricity.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1336.88,NaN
1,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1591.50,NaN
2,Afghanistan,Electricity - Gross production,2018.0,"Kilowatt-hours, million",1265.72,NaN
3,Afghanistan,Electricity - Gross production,2017.0,"Kilowatt-hours, million",1420.04,NaN
4,Afghanistan,Electricity - Gross production,2016.0,"Kilowatt-hours, million",1393.90,NaN


In [4]:
nuclear_electricity.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Argentina,Electricity - total nuclear production,2020.0,"Kilowatt-hours, million",10706.824,NaN
1,Argentina,Electricity - total nuclear production,2019.0,"Kilowatt-hours, million",7927.000,NaN
2,Argentina,Electricity - total nuclear production,2018.0,"Kilowatt-hours, million",6453.000,NaN
3,Argentina,Electricity - total nuclear production,2017.0,"Kilowatt-hours, million",6088.000,NaN
4,Argentina,Electricity - total nuclear production,2016.0,"Kilowatt-hours, million",8285.000,1.0


In [15]:
total_electricity = total_electricity.drop(columns=['Quantity Footnotes'])
nuclear_electricity = nuclear_electricity.drop(columns=['Quantity Footnotes'])
print('Total Electricity:\n',total_electricity.dtypes,'\n\nNuclear Electricity:\n',nuclear_electricity.dtypes)

Total Electricity:
 Country or Area             object
Commodity - Transaction     object
Year                       float64
Unit                        object
Quantity                   float64
dtype: object 

Nuclear Electricity:
 Country or Area             object
Commodity - Transaction     object
Year                       float64
Unit                        object
Quantity                   float64
dtype: object


,Country or Area,Commodity - Transaction,Year,Unit,Quantity
0,Afghanistan,Electricity - Gross production,2020.0,"Kilowatt-hours, million",1336.88
1,Afghanistan,Electricity - Gross production,2019.0,"Kilowatt-hours, million",1591.50
2,Afghanistan,Electricity - Gross production,2018.0,"Kilowatt-hours, million",1265.72
3,Afghanistan,Electricity - Gross production,2017.0,"Kilowatt-hours, million",1420.04
4,Afghanistan,Electricity - Gross production,2016.0,"Kilowatt-hours, million",1393.90
...,...,...,...,...,...
99784,Malaysia,Electricity - Gross production,2017.0,"Kilowatt-hours, million",164509.00
99785,Malaysia,Electricity - Gross production,2016.0,"Kilowatt-hours, million",156665.00
99786,Malaysia,Electricity - Gross production,2015.0,"Kilowatt-hours, million",150123.00
100000,fnSeqID,Footnote,NaN,NaN,NaN
